In [1]:
using DelimitedFiles

### Search in an array function

In [2]:
function searchsortednearest(a,x)
   idx = searchsortedfirst(a,x)
   if (idx==1); return idx; end
   if (idx>length(a)); return length(a); end
   if (a[idx]==x); return idx; end
   if (abs(a[idx]-x) < abs(a[idx-1]-x))
      return idx
   else
      return idx-1
   end
end

searchsortednearest (generic function with 1 method)

### Import of parameters to be used, also the convertion factors 
### Wavelength and distance also here

In [3]:
ang = readdlm("../pmt_ang_interp.txt",skipstart=1); # angular acceptance data
qe = readdlm("../pmt_qe_interp.txt",skipstart=1); # quantum efficiency data
qe[:,2]=qe[:,2]*0.01
wabs = readdlm("../water_abs_interp.txt",skipstart=1); # absorption length data
wscatt = readdlm("../water_scatt_rayleight.txt",skipstart=1); # scattering length data
b=0.835 # parameter b of the scattering
lam = 446; #Wavelength (nm)
xa = wabs[searchsortednearest(wabs[1:end,1],lam),2]; # absorption length
xs = wscatt[searchsortednearest(wscatt[1:end,1],lam),2]; # scattering length
Pq = qe[searchsortednearest(qe[1:end,1],lam),2]; # quantum efficiency
f=(1. /(4. *pi))*(3. /(3. +b))
human = 1; # Human eye conversion factor
lum = 1; # Brightness (lm)
dist = 400; # Distance

In [10]:
ang[1000:1050,:]

51×2 Array{Float64,2}:
 -0.25075  0.0     
 -0.25     0.0     
 -0.24875  0.0001  
 -0.2475   0.0002  
 -0.24625  0.0003  
 -0.245    0.0004  
 -0.24375  0.0005  
 -0.2425   0.0006  
 -0.24125  0.0007  
 -0.24     0.0008  
 -0.23875  0.0009  
 -0.2375   0.001   
 -0.23625  0.0011  
  ⋮                
 -0.2025   0.0038  
 -0.20125  0.0039  
 -0.2      0.004   
 -0.19875  0.004375
 -0.1975   0.00475 
 -0.19625  0.005125
 -0.195    0.0055  
 -0.19375  0.005875
 -0.1925   0.00625 
 -0.19125  0.006625
 -0.19     0.007   
 -0.18875  0.007375

### Integration for the setup of the semisphere for determined wavelength and distance to pmt

In [4]:
#one Lumen at 555 nm = 4,11·10^15 Photons per second  
Aeff3inch = 0.004536; # Effective area 3inch PMT [m^2]
ratdef = 4.11e15; #photon rate 1 lumen
#ratdef = 9e18; # photon rate for the ROV

ctmin = -1.;
ctmax = 0#cos(pi/6.)#0;
Nbct = 1000;
delct=(ctmax-ctmin)/Nbct;

ndist = 500; # steps over 500m
nmet  = 1; # step width (m)
rate = 0;
part = 0;


#loop over distance bins
ndist = 500 # steps over 500m
nmet = 1 #step width

for j = 0:ndist-1
    xm = (j+0.05)*nmet; # mean distance of bin
    Psca = nmet/xs*exp(-xm/xs); # probability of scattering inside the bin
    for l=0:Nbct-1
        ct = ctmin + (l+0.5)*delct;
        y=sqrt(dist*dist+xm*xm-2. *dist*xm*ct); # distance from scattering to PMT
        Pabs = exp(-(y+xm)/xa); # probability of non-absorption on full photon path
        cosa = (-dist*dist + xm*xm + y*y)/(2. *xm*y); # scattering angle
        cosb = (-xm*xm + y*y + dist*dist)/(2. *y*dist); # PMT-photon angle
        # included sanity check in angles
        if (abs((acos(ct)+acos(cosa)+acos(cosb))/pi-1)>1e-6) 
            println(ct, " ", cosa, " ", cosb)
        end
        PSAng = f*(1. +b*(cosa)^2.) # probability for angular scattering
        PxAng = ang[searchsortednearest(ang[1:end,1],cosb),2] # probability for photon on PMT
        rate += ratdef * human * lum * Aeff3inch/y/y * Pabs * Psca * PSAng * PxAng * Pq / Nbct;
        part += 0;
    end
            #println("area= ",Aeff3inch/y/y ,"Pabs= ", Pabs ,"Psca= ", Psca ,"PSAng= ", PSAng ,"PxAng= ", PxAng , "Pq/Nbct= ",Pq / Nbct)

end
println("Rate ", rate, "Hz")        

Rate 1031.1855162957352Hz


In [5]:
#one Lumen at 555 nm = 4,11·10^15 Photons per second  
Aeff3inch = 0.004536; # Effective area 3inch PMT [m^2]
ratdef = 4.11e15; #photon rate 1 lumen
ratdef = 9e18; # photon rate for the ROV
ratdef = 8*10^17; # photon rate for the ROV

ctmin = -1.;
ctmax = -cos(pi/6.)#0;
Nbct = 1000;
delct=(ctmax-ctmin)/Nbct;

ndist = 500; # steps over 500m
nmet  = 1; # step width (m)
rate = 0;
part = 0;


#loop over distance bins
ndist = 500 # steps over 500m
nmet = 1 #step width

for j = 0:ndist-1
    xm = (j+0.05)*nmet; # mean distance of bin
    Psca = nmet/xs*exp(-xm/xs); # probability of scattering inside the bin
    for l=0:Nbct-1
        ct = ctmin + (l+0.5)*delct;
        y=sqrt(dist*dist+xm*xm-2. *dist*xm*ct); # distance from scattering to PMT
        Pabs = exp(-(y+xm)/xa); # probability of non-absorption on full photon path
        cosa = (-dist*dist + xm*xm + y*y)/(2. *xm*y); # scattering angle
        cosb = (-xm*xm + y*y + dist*dist)/(2. *y*dist); # PMT-photon angle
        pmtcos = cos((0.)*pi/6.) # PMT rotation angle
        domcos = cosb*pmtcos+sqrt(1-cosb^2.)*sqrt(1-pmtcos^2.) # angle sum of rotation and arrival
        # included sanity check in angles
        if (abs((acos(ct)+acos(cosa)+acos(cosb))/pi-1)>1e-6) 
            println(ct, " ", cosa, " ", cosb)
        end
        PSAng = f*(1. +b*(cosa)^2.) # probability for angular scattering
        PxAng = ang[searchsortednearest(ang[1:end,1],domcos),2] # probability for photon on PMT
        rate += ratdef * human * lum * Aeff3inch/y/y * Pabs * Psca * PSAng * PxAng * Pq / Nbct;
        part += 0;
    end
            #println("area= ",Aeff3inch/y/y ,"Pabs= ", Pabs ,"Psca= ", Psca ,"PSAng= ", PSAng ,"PxAng= ", PxAng , "Pq/Nbct= ",Pq / Nbct)

end
println("Rate ", rate, "Hz")        

Rate 213605.97107927914Hz


### Code that attempts to do the DOM numbers of the slides presentation
### Change "sector" to change the PMT orientation

In [18]:
#one Lumen at 555 nm = 4,11·10^15 Photons per second  
Aeff3inch = 0.004536; # Effective area 3inch PMT [m^2]
ratdef = 4.11e15; #photon rate 1 lumen
#ratdef = 8e17; # photon rate for the ROV
#ratdef = 9*10^18; # photon rate for the ROV

ctmin = -1.;
ctmax = -cos(pi/6);#0;
Nbct = 100;
delct=(ctmax-ctmin)/Nbct;

ndist = 1000; # steps over 500m
nmet  = 5; # step width (m)
rate = 0;
part = 0;

ratesum = 0; # sum of individual rates
#loop over distance bins
ndist = 500 # steps over 500m
nmet = 1 #step width
pmtsteps=80#100 #number of steps to average the value
sector=[4.]#[0.,1.,2.,3.,4.,5.] # what part of the hemisphere you are looking
pmtcosb=zeros(pmtsteps)
for kk = 1:6
for ii =1:pmtsteps
    pmtcosb[ii] = cos((ii/pmtsteps+sector[kk])*pi/6.) # PMT rotation angle
end
for jj =1:pmtsteps
    rate = 0;
    for j = 0:ndist-1
        xm = (j+0.05)*nmet; # mean distance of bin
        Psca = nmet/xs*exp(-xm/xs); # probability of scattering inside the bin
        for l=0:Nbct-1
            ct = ctmin + (l+0.5)*delct;
            y=sqrt(dist*dist+xm*xm-2. *dist*xm*ct); # distance from scattering to PMT
            Pabs = exp(-(y+xm)/xa); # probability of non-absorption on full photon path
            cosa = (-dist*dist + xm*xm + y*y)/(2. *xm*y); # scattering angle
            cosb = (-xm*xm + y*y + dist*dist)/(2. *y*dist); # PMT-photon angle
            pmtcos = pmtcosb[jj] # PMT rotation angle
            domcos = cosb*pmtcos+sqrt(1-cosb^2.)*sqrt(1-pmtcos^2.) # angle sum of rotation and arrival
            # included sanity check in angles
            if (abs((acos(ct)+acos(cosa)+acos(cosb))/pi-1)>1e-6) 
                println(ct, " ", cosa, " ", cosb)
            end
            PSAng = f*(1. +b*(cosa)^2.) # probability for angular scattering
            PxAng = ang[searchsortednearest(ang[1:end,1],domcos),2] # probability for photon on PMT
            rate += ratdef * human * lum * Aeff3inch/y/y * Pabs * Psca * PSAng * PxAng * Pq / Nbct;
            part += 0;
        end
    end
    println(rate,"\t",pmtcosb[jj])
    ratesum += rate
end

println("For the sector= ", sector[kk], ", Rate ", ratesum/pmtsteps, "Hz")  
ratesum = 0
end


5.390141628110298e-10	-0.5056573733779844
6.28116473214832e-11	-0.5112930860770519
0.0	-0.5169068966820275
0.0	-0.5224985647159488
0.0	-0.528067850650368
0.0	-0.5336145159156115
0.0	-0.5391383229110003
0.0	-0.5446390350150268
0.0	-0.5501164165954934
0.0	-0.5555702330196023
0.0	-0.56100025066401
0.0	-0.566406236924833
0.0	-0.5717879602276121
0.0	-0.5771451900372335
0.0	-0.582477696867802
0.0	-0.587785252292473
0.0	-0.5930676289532374
0.0	-0.5983246005706586
0.0	-0.6035559419535711
0.0	-0.6087614290087207
0.0	-0.6139408387503665
0.0	-0.6190939493098341
0.0	-0.6242205399450175
0.0	-0.6293203910498373
0.0	-0.6343932841636454
0.0	-0.6394390019805847
0.0	-0.6444573283588974
0.0	-0.6494480483301835
0.0	-0.6544109481086103
0.0	-0.6593458151000688
0.0	-0.6642524379112817
0.0	-0.6691306063588582
0.0	-0.6739801114782976
0.0	-0.6788007455329415
0.0	-0.6835923020228714
0.0	-0.6883545756937541
0.0	-0.693087362545636
0.0	-0.69779045984168
0.0	-0.7024636661168516
0.0	-0.7071067811865475
0.0	-0.7117196

BoundsError: BoundsError: attempt to access 1-element Array{Float64,1} at index [2]

In [16]:
pmtsteps=100 #number of steps to average the value
sector=3. # what part of the hemisphere you are looking
pmtcosb=zeros(pmtsteps)
for ii =1:pmtsteps
    pmtcosb[ii] = acos(cos((ii/pmtsteps+sector)*pi/6.)) # PMT rotation angle
end
pmtcosb/pi

100-element Array{Float64,1}:
 0.5016666666666666
 0.5033333333333333
 0.505             
 0.5066666666666666
 0.5083333333333333
 0.51              
 0.5116666666666666
 0.5133333333333333
 0.515             
 0.5166666666666667
 0.5183333333333333
 0.5199999999999999
 0.5216666666666666
 ⋮                 
 0.6483333333333333
 0.65              
 0.6516666666666667
 0.6533333333333333
 0.6550000000000001
 0.6566666666666666
 0.6583333333333333
 0.66              
 0.6616666666666666
 0.6633333333333333
 0.665             
 0.6666666666666666

In [162]:
1+9e18

9.0e18

In [163]:
1+9*(10^(18))

9000000000000000001

In [173]:
-cos(pi/6)

-0.8660254037844387